In [ ]:
import tensorflow as tf
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpim
import numpy as np
import random
import cv2

training_path = "fruits-360_dataset/Training"
test_path = "fruits-360_dataset/Test"

try:
    STATS = np.load("stats.npy", allow_pickle=True)
except FileNotFoundError as fnf:
    print("Not found stats file.")
    STATS = []

# Parameters    
GRAY_SCALE = False
FRUITS = os.listdir(training_path)

random.shuffle(FRUITS)
FRUITS = FRUITS[:15] 


In [ ]:
def load_data(directory_path, fruit_restriction=None):
    data = []
    labels = []
               
            
    for fruit_name in FRUITS:
        class_num = FRUITS.index(fruit_name)                
        
        path = os.path.join(directory_path, fruit_name)
        print(f"Adding: {class_num:>3} {fruit_name:<20}, {path}")
        
        for img in os.listdir(path):
            img_path = os.path.join(path, img)
            
            if GRAY_SCALE:
                image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            else:
                image = mpim.imread(img_path)
                
            image = image / 255.0
                    
            data.append([image, class_num])

#         print(FRUITS[class_num])
#         plt.figure()    
#         plt.imshow(image, cmap='gray')
#         plt.show()
        
    random.shuffle(data)
        
    X = []
    y = []
    
    
    for image, label in data:
        X.append(image)
        y.append(label)
    
#     X = data[:, 0]
#     y = data[:, 1]
    X = np.array(X)
    y = np.array(y)
    
    if GRAY_SCALE:
        print("Reshaping gray scale")
        X = X.reshape(-1, X.shape[1], X.shape[2], 1)
        
    return X, y      

In [ ]:
X_training, y_training = load_data(training_path)
print("\nCreated training array")    
print(f"X shape: {X_training.shape}")
print(f"y shape: {y_training.shape}")

In [ ]:
X_test, y_test = load_data(test_path)
print("\nCreated test array")    
print(f"X shape: {X_test.shape}")
print(f"y shape: {y_test.shape}")

In [ ]:
# Limit gpu memory usage
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.2
sess = tf.compat.v1.Session(config=config)

In [ ]:
from tensorflow.keras.layers import Dense, Flatten, Conv2D, Conv3D, MaxPooling2D, MaxPooling3D, Activation, Dropout

pic_shape = X_training.shape[1:]
label_count = len(FRUITS)

print(f"Pic shape: {pic_shape}")

# Input
model = tf.keras.models.Sequential([
    Conv2D(64, (3, 3), activation='relu', input_shape=pic_shape),
    MaxPooling2D(pool_size=(2, 2)),
    
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    
    Dropout(0.2),
    
    Flatten(),
    Dense(32, activation='relu'),
    Dense(label_count, activation='softmax')    
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

In [ ]:
model.fit(X_training, y_training, batch_size=20, epochs=3)

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
predictions = model.predict(X_test)

print(f"Test Accuracy: {accuracy:>2.4f}")
print(f"Test loss: {loss:>2.4f}")

count_incorrect = 0
for x, predict in enumerate(predictions):
    class_num = np.argmax(predict)
    valid = y_test[x] == class_num
    if not valid:
        count_incorrect += 1
print(f"Incorrect predictions: {count_incorrect}")       
